<a href="https://colab.research.google.com/github/Souleymaneawe/Souleymaneawe/blob/main/Seasonal_Food_Recommandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Étape 1 : Installer les bibliothèques
!pip install gradio google-generativeai firebase-admin

# Étape 2 : Importer les bibliothèques
import gradio as gr
import google.generativeai as genai
import firebase_admin
from firebase_admin import credentials, auth, firestore
from google.colab import userdata
import json

In [17]:
# Étape 3 : Configurer Firebase
try:
    # Check if a default app already exists
    if not firebase_admin._apps:
        firebase_config = {
            "type": userdata.get('FIREBASE_TYPE'),
            "project_id": userdata.get('FIREBASE_PROJECT_ID'),
            "private_key_id": userdata.get('FIREBASE_PRIVATE_KEY_ID'),
            "private_key": userdata.get('FIREBASE_PRIVATE_KEY').replace('\\n', '\n'),
            "client_email": userdata.get('FIREBASE_CLIENT_EMAIL'),
            "client_id": userdata.get('FIREBASE_CLIENT_ID'),
            "auth_uri": userdata.get('FIREBASE_AUTH_URI'),
            "token_uri": userdata.get('FIREBASE_TOKEN_URI'),
            "auth_provider_x509_cert_url": userdata.get('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
            "client_x509_cert_url": userdata.get('FIREBASE_CLIENT_X509_CERT_URL')
        }
        cred = credentials.Certificate(firebase_config)
        firebase_admin.initialize_app(cred)
    db = firestore.client()
except userdata.SecretNotFoundError:
    print("Erreur : Identifiants Firebase non trouvés. Ajoutez les clés dans les Secrets de Colab.")
    firebase_config = None
    db = None

In [10]:
# Étape 4 : Configurer l'API Gemini
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
except userdata.SecretNotFoundError:
    GEMINI_API_KEY = None
    print("Erreur : Clé API Gemini non trouvée. Ajoutez 'GEMINI_API_KEY' dans les Secrets de Colab.")


In [18]:
# Étape 5 : Charger les données des saisons depuis JSON
def load_seasons():
    try:
        with open("seasons.json", "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return {
            "seasons": [
                {"name": "Printemps", "description": "Saison des légumes frais et des plats légers."},
                {"name": "Été", "description": "Saison des fruits juteux et des repas rafraîchissants."},
                {"name": "Automne", "description": "Saison des récoltes riches et des plats réconfortants."},
                {"name": "Hiver", "description": "Saison des soupes chaudes et des plats copieux."}
            ]
        }

In [19]:
# Étape 6 : Fonctions Firebase
def register_user(email, password):
    try:
        user = auth.create_user(email=email, password=password)
        db.collection('users').document(user.uid).set({'email': email, 'preferences': {}})
        return f"Inscription réussie pour {email}."
    except Exception as e:
        return f"Erreur lors de l'inscription : {str(e)}"

def login_user(email, password):
    try:
        # Vérification manuelle (Firebase ne fournit pas de méthode directe pour vérifier le mot de passe)
        user = auth.get_user_by_email(email)
        return user.uid, f"Connexion réussie pour {email}."
    except Exception as e:
        return None, f"Erreur lors de la connexion : {str(e)}"

def save_preferences(uid, preferences):
    try:
        db.collection('users').document(uid).update({'preferences': preferences})
        return "Préférences enregistrées avec succès."
    except Exception as e:
        return f"Erreur lors de l'enregistrement des préférences : {str(e)}"

In [20]:
# Étape 7 : Fonction pour les suggestions de plats
def get_food_suggestions(season, preferences, uid=None):
    if not GEMINI_API_KEY:
        return "Erreur : Clé API Gemini manquante."
    model = genai.GenerativeModel("gemini-1.5-pro")
    prompt = f"Suggère des plats typiques de la saison {season}. Considère les préférences alimentaires suivantes : {preferences}."
    try:
        response = model.generate_content(prompt)
        if uid:
            save_preferences(uid, preferences)
        return response.text
    except Exception as e:
        return f"Erreur API : {str(e)}"

In [21]:
# Étape 8 : Interface Gradio
with gr.Blocks(
    title="Suggestions de plats saisonniers avec Firebase",
    css="""
        .gradio-container { background-color: #f5f7fa; font-family: 'Segoe UI', Arial, sans-serif; }
        h1 { color: #2c3e50; text-align: center; font-size: 2em; margin-bottom: 20px; }
        .gr-button { background-color: #28a745; color: white; border-radius: 10px; padding: 10px; }
        .gr-button:hover { background-color: #218838; }
        .gr-textbox, .gr-dropdown { border-radius: 10px; border: 1px solid #ced4da; padding: 8px; }
        .gr-form { background-color: #ffffff; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
    """
) as app:
    gr.Markdown("# Suggestions de plats saisonniers")

    # Section d'authentification
    with gr.Tab("Connexion / Inscription"):
        gr.Markdown("## Connexion ou Inscription")
        email = gr.Textbox(label="Adresse e-mail")
        password = gr.Textbox(label="Mot de passe", type="password")
        login_btn = gr.Button("Se connecter")
        register_btn = gr.Button("S'inscrire")
        auth_output = gr.Textbox(label="Statut de l'authentification")
        uid_state = gr.State()  # Stocker l'UID de l'utilisateur connecté

        login_btn.click(
            fn=login_user,
            inputs=[email, password],
            outputs=[uid_state, auth_output]
        )
        register_btn.click(
            fn=register_user,
            inputs=[email, password],
            outputs=auth_output
        )

    # Section des suggestions
    with gr.Tab("Suggestions de plats"):
        seasons_data = load_seasons()
        season_names = [season["name"] for season in seasons_data["seasons"]]
        gr.Markdown("## Sélectionnez une saison")
        season = gr.Dropdown(choices=season_names, label="Saison")
        season_description = gr.Textbox(label="Description de la saison", interactive=False)

        def update_season_description(selected_season):
            for season_data in seasons_data["seasons"]:
                if season_data["name"] == selected_season:
                    return season_data["description"]
            return "Saison non trouvée."

        season.change(fn=update_season_description, inputs=season, outputs=season_description)

        preferences = gr.Textbox(label="Préférences alimentaires (ex. végétarien, sans gluten)")
        suggest_btn = gr.Button("Obtenir des suggestions")
        output = gr.Textbox(label="Suggestions de plats", lines=5)
        suggest_btn.click(
            fn=get_food_suggestions,
            inputs=[season, preferences, uid_state],
            outputs=output
        )

In [22]:
# Étape 9 : Instructions pour exécuter
app.launch(share=True)
# Un lien public sera généré pour accéder à l'interface via Gradio Live


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2ded37be4723a0ba65.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
